I would like to process text in the most parallel possible way.



References:
 + Related questions online:
     + Parallization using sklearn and tfidfvectorizer: https://stackoverflow.com/questions/28396957/sklearn-tfidf-vectorizer-to-run-as-parallel-jobs   

 +  Really good answer to parallelize using dataframes in python: http://blog.adeel.io/2016/11/06/parallelize-pandas-map-or-apply/
  To re-do it using joblib: https://joblib.readthedocs.io/en/latest/auto_examples/parallel_memmap.html 

In [1]:
# Some libraries
#!pip install git+https://github.com/uqfoundation/pathos
from  time import time
from sklearn.datasets import fetch_20newsgroups
import pathos.multiprocessing
import pandas as pd
import numpy as np
from pathos.multiprocessing import ProcessingPool as Pool
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse as sp

In [2]:


# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))

print("done in %0.3fs." % (time() - t0))


Loading dataset...
done in 1.211s.


In [3]:
import pandas as pd
# I put the data in pandas dataframe because of convinience for
# other cases I am working with
data_pd = pd.DataFrame(dataset.data)
data_pd.rename(columns = {0:'text'},inplace = True)
data_pd.head()

,text
0,Well i'm not sure about the story nad it did s...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re..."
2,Although I realize that principle is not one o...
3,Notwithstanding all the legitimate fuss about ...
4,"Well, I will have to change the scoring on my ..."


In [4]:
# To give some more drama let artificiallyt growth the database
data_pd = pd.concat([data_pd,data_pd])
data_pd = pd.concat([data_pd,data_pd])
data_pd = pd.concat([data_pd,data_pd])
data_pd = pd.concat([data_pd,data_pd])

In [5]:
# Use tf-idf features
# THE FIT IS NOT EASY TO PARALLELIZE
print("Extracting tf-idf features")
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit(data_pd['text'])
print("done in %0.3fs." % (time() - t0))

Extracting tf-idf features
done in 19.646s.


In [6]:
# To select the feature in not easy to parallelize but
# after been done, you can parallelize the actual transformation,
# this is, the vectorization of the process
# Use tf-idf TRANSFORM.
print("Transforming tf-idf features...")
tfidf = tfidf_vectorizer.transform(data_pd['text'])
print("done in %0.3fs." % (time() - t0))

Transforming tf-idf features...
done in 39.631s.


In [7]:
%%time
# I mostly borry this code from http://blog.adeel.io/2016/11/06/parallelize-pandas-map-or-apply/
#ValueError: Pool not running
import multiprocessing
import pandas as pd
import numpy as np
import scipy.sparse as sp
#num_partitions = 5
num_cores = multiprocessing.cpu_count()
num_partitions = num_cores-2 # I like to leave some cores for other
#processes
print(num_partitions)
from pathos.multiprocessing import ProcessingPool as Pool
def parallelize_dataframe(df, func):
    #from pathos.multiprocessing import ProcessingPool as Pool
    a = np.array_split(df, num_partitions)
    del df
    pool = Pool(num_cores)
    #df = pd.concat(pool.map(func, [a,b,c,d,e]))
    df = sp.vstack(pool.map(func, a), format='csr')
    #pool.close() # For multiprocessing library
    #pool.join() # For multiprocessing library
    pool.terminate() # For pathos.multiprocessing library
    pool.restart() # For pathos.multiprocessing library
    return df
 
def test_func(data):
    #print("Process working on: ",data)
    tfidf_matrix = tfidf_vectorizer.transform(data["text"])
    #return pd.DataFrame(tfidf_matrix.toarray())
    return tfidf_matrix
 
#df = pd.DataFrame({'col': [0,1,2,3,4,5,6,7,8,9]})
#df =  data_pd
tfidf_parallel = parallelize_dataframe(data_pd, test_func)
#print(tfidf_parallel)

6
CPU times: user 635 ms, sys: 304 ms, total: 939 ms
Wall time: 6.03 s


In [8]:
# Are really the same?
# You could visualize them, here I am going to compare the sum of all their elements.
tfidf_parallel.sum()

1035350.6785209675

In [9]:
tfidf.sum()

1035350.6785209675